In [123]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error


In [124]:
train_data = pd.read_csv('./train.csv', parse_dates=['month'])
test_data = pd.read_csv('./test.csv', parse_dates=['month'])
au_train_data = pd.read_csv('auxiliary-data/comCountTrain.csv')
au_test_data = pd.read_csv('auxiliary-data/comCountTest.csv')

In [125]:
train_data.drop(['block', 'eco_category', 'elevation', 'planning_area'], axis=1, inplace=True)
train_data

,month,town,flat_type,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,latitude,longitude,subzone,region,resale_price
0,2001-08-01,pasir ris,4 room,pasir ris drive 4,01 to 03,118.0,model a,1989,1.369008,103.958697,pasir ris drive,east region,209700.0
1,2014-10-01,punggol,5-room,punggol field,10 to 12,110.0,improved,2003,1.399007,103.906991,punggol field,north-east region,402300.0
2,2020-09-01,sengkang,5 room,fernvale lane,01 to 03,112.0,premium apartment,2004,1.388348,103.873815,fernvale,north-east region,351000.0
3,2000-10-01,clementi,3 room,clementi avenue 4,07 to 09,67.0,new generation,1980,1.318493,103.766702,clementi north,west region,151200.0
4,2013-01-01,bukit batok,3-room,bukit batok street 11,07 to 09,73.0,model a,1985,1.348149,103.742658,bukit batok west,west region,318600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431727,2005-03-01,woodlands,4 room,Woodlands Drive 16,01 to 03,101.0,model a,2000,1.429658,103.792583,woodlands south,north region,238500.0
431728,2016-04-01,sengkang,4 room,fernvale road,13 to 15,95.0,premium apartment,2012,1.390053,103.875941,fernvale,north-east region,376200.0
431729,2011-01-01,tampines,3-room,tampines street 81,01 to 03,67.0,new generation,1986,1.349224,103.934913,tampines west,east region,255600.0
431730,2013-05-01,sengkang,5-room,compassvale walk,16 to 18,123.0,improved,1999,1.389941,103.900721,sengkang town centre,north-east region,508500.0


In [126]:
# There are multiple duplicates of ('latitude', 'longitude')
train_data.groupby(['latitude', 'longitude']).size()

latitude  longitude 
1.270380  103.823236     58
1.270919  103.822685     62
1.271409  103.810888    144
1.271463  103.825684     55
1.271691  103.809852     18
                       ... 
1.456235  103.814292     21
1.456425  103.815858     93
1.456474  103.817181     59
1.456546  103.816764     60
1.457071  103.815308    113
Length: 9138, dtype: int64

In [127]:
test_data.drop(['block', 'eco_category', 'elevation', 'planning_area'], axis=1, inplace=True)
# test_data

In [128]:
au_test_data = au_test_data.loc[:, ["commercialCount_5","marketCount_3","shoppingCount_3","stationCount_2"]]
# au_test_data

In [129]:
au_train_data = au_train_data.loc[:, ["commercialCount_5","marketCount_3","shoppingCount_3","stationCount_2"]]
# au_train_data

In [130]:
com_train_data = pd.concat([train_data, au_train_data], axis=1, ignore_index=False)
# com_train_data

In [131]:
com_test_data = pd.concat([test_data, au_test_data], axis=1, ignore_index=False)
# com_test_data

# Data Preprocessing

In [132]:
value_counts = com_test_data['flat_type'].value_counts()
# value_counts

In [133]:
# convert flat_type to int
def process_value(value):
    if value.startswith('e'):
        return 6
    elif value.startswith('m'):
        return 7
    else:
        return int(value[0])
    

com_train_data['flat_type'] = com_train_data['flat_type'].apply(process_value)
com_test_data['flat_type'] = com_test_data['flat_type'].apply(process_value)
value_counts = com_test_data['flat_type'].value_counts()
# value_counts

In [134]:
value_counts = com_test_data['storey_range'].value_counts()
# value_counts

In [135]:
# storey_range process
def transfer(x):
    storeys = x.split(' to ')
    return int(int(storeys[0])+int(storeys[1]))/2


com_train_data['storey_range'] = com_train_data['storey_range'].apply(transfer)
com_test_data['storey_range'] = com_test_data['storey_range'].apply(transfer)
value_counts = com_test_data['storey_range'].value_counts()
# value_counts

In [136]:
# convert time to float
com_train_data['sell_time'] = com_train_data['month'].apply(lambda x: x.year + x.month / 12)
com_train_data.drop('month', axis=1, inplace=True)
com_test_data['sell_time'] = com_test_data['month'].apply(lambda x: x.year + x.month / 12)
com_test_data.drop('month', axis=1, inplace=True)
# com_train_data

In [137]:
# add age data
age_train_data = pd.read_csv('WithAge.csv')
age_train_data = age_train_data.loc[:, ["0-14", "15-29", "30-59", "60+"]]
all_train_data = pd.concat([com_train_data, age_train_data], axis=1, ignore_index=False)

age_test_data = pd.read_csv("TestWithAge.csv")
age_test_data = age_test_data.loc[:, ["0-14", "15-29", "30-59", "60+"]]
all_test_data = pd.concat([com_test_data, age_test_data], axis=1, ignore_index=False)


In [138]:
# 找出包含缺失值的行
missing_rows = age_train_data.isna().any(axis=1)

# 显示包含缺失值的行
data_with_missing_values = age_train_data[missing_rows]
data_with_missing_values

,0-14,15-29,30-59,60+
7169,NaN,NaN,NaN,NaN
15055,NaN,NaN,NaN,NaN
19665,NaN,NaN,NaN,NaN
26727,NaN,NaN,NaN,NaN
27592,NaN,NaN,NaN,NaN
...,...,...,...,...
422795,NaN,NaN,NaN,NaN
424835,NaN,NaN,NaN,NaN
426748,NaN,NaN,NaN,NaN
427380,NaN,NaN,NaN,NaN


In [139]:
all_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431732 entries, 0 to 431731
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   town                 431732 non-null  object 
 1   flat_type            431732 non-null  int64  
 2   street_name          431732 non-null  object 
 3   storey_range         431732 non-null  float64
 4   floor_area_sqm       431732 non-null  float64
 5   flat_model           431732 non-null  object 
 6   lease_commence_date  431732 non-null  int64  
 7   latitude             431732 non-null  float64
 8   longitude            431732 non-null  float64
 9   subzone              431732 non-null  object 
 10  region               431732 non-null  object 
 11  resale_price         431732 non-null  float64
 12  commercialCount_5    431732 non-null  int64  
 13  marketCount_3        431732 non-null  int64  
 14  shoppingCount_3      431732 non-null  int64  
 15  stationCount_2   

In [140]:
# use all_train_data and all_test_data

all_train_data = all_train_data.dropna()
all_test_data = all_test_data.dropna()

# 对分类特征进行编码
categorical_features = ['town', 'street_name', 'flat_model', 'subzone', 'region']
for col in categorical_features:
    lbl = LabelEncoder()
    all_train_data[col] = lbl.fit_transform(all_train_data[col])


# 对all_test_data的分类特征进行编码，与训练数据集保持一致
for col in categorical_features:
    lbl = LabelEncoder()
    all_test_data[col] = lbl.fit_transform(all_test_data[col])

# Run Model and Predict

## Evaluation and Test predict

In [141]:
def evaluate(model, train, test):
    # 将数据分为特征和目标变量
    X = train.drop('resale_price', axis=1)
    y = train['resale_price']

    # 将数据集分为训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 定义要搜索的参数网格
    tree_param_grid = {
            'n_estimators': [7000, 9000],
            'learning_rate': [0.1,],
        #     'num_leaves': [31, 50],
            'max_depth': [None, 15, 17, 19, 34],
        # 'min_child_samples': [20, 30]
    }
    
    linear_param_grid = {
        'fit_intercept': [True, False],
    }

    # 使用GridSearchCV搜索最佳参数
#     grid_search = GridSearchCV(model, tree_param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
    grid_search = GridSearchCV(model, linear_param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    # 输出最佳参数
    print("最佳参数：", grid_search.best_params_)

    # 使用最佳参数重新训练模型
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, y_train)

    # 进行预测
    y_pred = best_model.predict(X_test)

    # 计算并输出均方误差
    mse = mean_squared_error(y_test, y_pred)
    print("均方误差：", mse)
    
    # 使用训练好的模型进行预测
    test_pred = best_model.predict(test)

    # 为预测结果创建一个新的DataFrame
    result_df = pd.DataFrame({'Id': np.arange(len(test_pred)), 'Predicted': test_pred})

    # 将结果保存到CSV文件
    result_df.to_csv('predictions.csv', index=False)

## LightGBM

In [142]:
# 创建LightGBM回归器
# lgbm = LGBMRegressor()

# evaluate(lgbm, all_train_data, all_test_data)

## Linear Regression

In [143]:
# 找出包含缺失值的行
missing_rows = all_train_data.isna().any(axis=1)

# 显示包含缺失值的行
data_with_missing_values = all_train_data[missing_rows]
data_with_missing_values

,town,flat_type,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,latitude,longitude,subzone,...,resale_price,commercialCount_5,marketCount_3,shoppingCount_3,stationCount_2,sell_time,0-14,15-29,30-59,60+


In [145]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

evaluate(regressor, all_train_data, all_test_data)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
最佳参数： {'fit_intercept': True}
均方误差： 3484700396.823359


## PipelineCatBoost